# Decision tree

This popular and powerful mmodel is constructed by learning a set of simple decision rules inferred from the data features in order to predict the value of a target variable. It is used for classification and regression. Decision Trees are an example of non-parametric supervised machine learning method. 


### Decision tree - Classification

DTs are capeble of multi-class classification on a dataset. 

As with all supervised classification method, the input is the data set $D = \{(\mathbf{x}_i,\mathbf{y}_i)\}_{i=1}^{N} $ with features $\mathbf{x}_i \in \mathbb{R}^{n}$ and labels $\mathbf{y}_i \in \mathbb{R}^{l}$.

Let us check this example with the **iris** dataset (again!).

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.colors import ListedColormap

from sklearn.datasets import load_iris
from sklearn import tree
iris = load_iris() # load the dataset
print(iris.DESCR)  # provides a description of the dataset

# build the decision tree
# complete model
X, y = iris.data, iris.target
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X, y)

# plot - tree visualization
plt.figure(figsize=(10, 10))
tree.plot_tree(clf, proportion=False)
plt.title("iris dataset - Decision Tree classification")
plt.show()

In [ ]:
# the restricted model
X = iris.data[:,[0,1]]
y = iris.target
clf_res = tree.DecisionTreeClassifier()
clf_res = clf.fit(X, y)

# Now plot the decision boundary using a fine mesh as input to a
# filled contour plot
cmap = plt.cm.RdYlBu
x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
plot_step = 0.1
xx, yy = np.meshgrid(
            np.arange(x_min, x_max, plot_step), np.arange(y_min, y_max, plot_step)
        )
Z = clf_res.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

plt.figure()
plt.contourf(xx, yy, Z, cmap=cmap)
plt.scatter(
    X[:,0],
    X[:,1],
    c=y,
    cmap=ListedColormap(["r", "y", "b"]),
    edgecolor="k",
    s=20,
)
plt.title("iris dataset - Decision Tree classification")
plt.show()

### Decision tree - Regression

DTs can also be use for regressions. 

Here we use the decision tree to fit a curve describing damped oscillations $\left(e^{-\frac{1}{2}x^2} \cos(3x)\right)$ with added noise.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.tree import DecisionTreeRegressor
from sklearn import tree

# The data - the function to fit
rng = np.random.RandomState(1234)
X = np.sort(5 * rng.rand(80, 1), axis=0)    # x-axis values
y = (np.exp(-0.5 * X) * np.cos(3 * X)).ravel()  # flattes the data of the function
y[::5] += 1 * (0.5 - rng.rand(16))      # places noise every 5 elements of data

# the model - regression
regr = DecisionTreeRegressor(max_depth=2)
regr.fit(X,y)

# predictions
X_test = np.arange(0.0, 5.0, 0.01)[:, np.newaxis] # test data
y_test = regr.predict(X_test)

# plot the results
plt.figure()
plt.scatter(X, y, s=20, edgecolor="black", c="darkorange", label="data")
plt.plot(X_test, y_test, color="cornflowerblue", label="max_depth=2", linewidth=2)
plt.xlabel("data")
plt.ylabel("target")
plt.title("Decision Tree Regression")
plt.legend()
plt.show()

## Overfitting

Decision Trees always overfit!

### Exercise

**Task:**
Build decision trees for the regression example changing the tree depth or maximum number of leaves.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.tree import DecisionTreeRegressor
from sklearn import tree

# The data - the function to fit
rng = np.random.RandomState(1234)
X = np.sort(5 * rng.rand(80, 1), axis=0)    # x-axis values
y = (np.exp(-0.5 * X) * np.cos(3 * X)).ravel()  # flattes the data of the function
y[::5] += 1 * (0.5 - rng.rand(16))      # places noise every 5 elements of data

# the model - regression
regr = DecisionTreeRegressor(max_depth=2)
regr.fit(X,y)
# models with different depth
regr_d1 = DecisionTreeRegressor(max_depth=5)
regr_d1.fit(X,y)
# models with different number of leaves
regr_nl1 = DecisionTreeRegressor(max_leaf_nodes=3)
regr_nl1.fit(X,y)

# predictions
X_test = np.arange(0.0, 5.0, 0.01)[:, np.newaxis] # test data
y_test = regr.predict(X_test)
y_test_d1 = regr_d1.predict(X_test)
y_test_nl1 = regr_nl1.predict(X_test)

# plot the results
plt.figure()
plt.scatter(X, y, s=20, edgecolor="black", c="darkorange", label="data")
plt.plot(X_test, y_test, color="cornflowerblue", label="max_depth=2", linewidth=2)
plt.plot(X_test, y_test_d1, color="yellowgreen", label="max_depth=5", linewidth=2)
plt.plot(X_test, y_test_nl1, color="red", label="max_leaf_nodes=3", linewidth=2)
plt.xlabel("data")
plt.ylabel("target")
plt.title("Decision Tree Regression")
plt.legend()
plt.show()

## Decision Tree - Structure

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

from sklearn import tree
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

# load the dataser
iris = load_iris()
X = iris.data
y = iris.target
# split the data into training and test (default test = 25%)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

# build a model with a maximum number of leaf nodes.
clf = DecisionTreeClassifier(max_leaf_nodes=3, random_state=0)
clf.fit(X_train, y_train)

# information on the model
n_nodes = clf.tree_.node_count # number of nodes
children_left = clf.tree_.children_left # ids of left child
children_right = clf.tree_.children_right # ids of right child
feature = clf.tree_.feature # feature used for the split
threshold = clf.tree_.threshold # threshold value for the split
values = clf.tree_.value # summary of the data

# identifying splitting from leaf nodes.
node_depth = np.zeros(shape=n_nodes, dtype=np.int64)
is_leaves = np.zeros(shape=n_nodes, dtype=bool)
stack = [(0, 0)]  # start with the root node id (0) and its depth (0)
while len(stack) > 0:
    # `pop` ensures each node is only visited once
    node_id, depth = stack.pop()
    node_depth[node_id] = depth

    # If the left and right child of a node is not the same we have a split
    # node
    is_split_node = children_left[node_id] != children_right[node_id]
    # If a split node, append left and right children and depth to `stack`
    # so we can loop through them
    if is_split_node:
        stack.append((children_left[node_id], depth + 1))
        stack.append((children_right[node_id], depth + 1))
    else:
        is_leaves[node_id] = True


# report on the tree structure
print(
    "The binary tree structure has {n} nodes and has "
    "the following tree structure:\n".format(n=n_nodes)
)
for i in range(n_nodes):
    if is_leaves[i]:
        print(
            "{space}node={node} is a leaf node with value={value}.".format(
                space=node_depth[i] * "\t", node=i, value=np.around(values[i], 3)
            )
        )
    else:
        print(
            "{space}node={node} is a split node with value={value}: "
            "go to node {left} if X[:, {feature}] <= {threshold} "
            "else to node {right}.".format(
                space=node_depth[i] * "\t",
                node=i,
                left=children_left[i],
                feature=feature[i],
                threshold=threshold[i],
                right=children_right[i],
                value=np.around(values[i], 3),
            )
        )

# tree visulaization
tree.plot_tree(clf, proportion=True)
plt.show()

# Random Forest

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import ListedColormap

from sklearn.datasets import load_iris, make_moons
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn import preprocessing

from sklearn.datasets import load_iris

In [ ]:
# load data
iris = load_iris() # load the datase
# print(iris.DESCR)  # provides a description of the dataset

plot_idx = 1    # initializing the plot labels

# Parameters
n_classes = 3 #
n_estimators = 1 # number of trees in the forest
plot_step = 0.02  # fine step width for decision surface contours
plot_step_coarser = 0.5  # step widths for coarse classifier guesses
RANDOM_SEED = 42  # fix the seed on each iteration

models = [
    DecisionTreeClassifier(max_depth=3),
    RandomForestClassifier(n_estimators=n_estimators)
]

plt.figure(figsize=(8,8))

# the restricted model
# We use different combinations of the pairs of features
for pair in ([0, 1], [0, 2], [2, 3]):
    for model in models:
        # We only take the two corresponding features
        X = iris.data[:, pair]
        y = iris.target

        # Shuffle
        idx = np.arange(X.shape[0])
        np.random.seed(RANDOM_SEED)
        np.random.shuffle(idx)
        X = X[idx]
        y = y[idx]

        # Standardize
        mean = X.mean(axis=0)
        std = X.std(axis=0)
        X = (X - mean) / std

        # Train
        model.fit(X, y)

        scores = model.score(X, y)
        # Create a title for each column and the console by using str() and
        # slicing away useless parts of the string
        model_title = str(type(model)).split(".")[-1][:-2][: -len("Classifier")]

        model_details = model_title
        if hasattr(model, "estimators_"):
            model_details += " with {} estimators".format(len(model.estimators_))
        print(model_details + " with features", pair, "has a score of", scores)

        plt.subplot(3, 2, plot_idx)
        if plot_idx <= len(models):
            # Add a title at the top of each column
            plt.title(model_title, fontsize=9)

        # filled contour plot
        x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
        y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
        xx, yy = np.meshgrid(
            np.arange(x_min, x_max, plot_step), np.arange(y_min, y_max, plot_step)
        )

        # Plot either a single DecisionTreeClassifier or alpha blend the
        # decision surfaces of the ensemble of classifiers
        if isinstance(model, DecisionTreeClassifier):
            Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
            Z = Z.reshape(xx.shape)
            cs = plt.contourf(xx, yy, Z, cmap=cmap)
        else:
            # Choose alpha blend level with respect to the number
            # of estimators
            estimator_alpha = 1.0 / len(model.estimators_)
            for tree in model.estimators_:
                Z = tree.predict(np.c_[xx.ravel(), yy.ravel()])
                Z = Z.reshape(xx.shape)
                cs = plt.contourf(xx, yy, Z, alpha=estimator_alpha, cmap=cmap)

        # Plot the training points, these are clustered together and have a
        # black outline
        plt.scatter(
            X[:, 0],
            X[:, 1],
            c=y,
            cmap=ListedColormap(["r", "y", "b"]),
            edgecolor="k",
            s=20,
        )
        plot_idx += 1  # move on to the next plot in sequence

plt.suptitle("Classifiers on feature subsets of the Iris dataset", fontsize=12)
plt.axis("tight")
plt.tight_layout(h_pad=0.2, w_pad=0.2, pad=2.5)
plt.show()


## Ejercicio

Clasificación con el datos de dos medias lunas

In [ ]:
# Generate the data
n_samples = 200
noise = 0.1
X_moons, y_moons = make_moons(n_samples=n_samples, noise=noise)
# Scale it to a range
min_max_scaler = preprocessing.MinMaxScaler()
X = min_max_scaler.fit_transform(X_moons)
y = y_moons